In [2]:
# Import dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop unnecessary columns
application_df = application_df.drop(columns=["EIN", "NAME"])

# Binning low-frequency categories
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = application_type_counts[application_type_counts < 500].index.tolist()
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].apply(lambda x: x if x not in application_types_to_replace else "Other")

classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = classification_counts[classification_counts < 1000].index.tolist()
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].apply(lambda x: x if x not in classifications_to_replace else "Other")

# One-hot encoding
application_df = pd.get_dummies(application_df)

In [4]:
# Split data into features and target
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values
y = application_df["IS_SUCCESSFUL"].values

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
# Define the optimized model
nn_opt = tf.keras.models.Sequential()
nn_opt.add(tf.keras.layers.Dense(units=100, input_dim=X_train_scaled.shape[1], activation="relu"))
nn_opt.add(tf.keras.layers.Dropout(0.2))  # Dropout layer
nn_opt.add(tf.keras.layers.Dense(units=50, activation="relu"))
nn_opt.add(tf.keras.layers.BatchNormalization())  # Batch normalization
nn_opt.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_opt.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [7]:
# Define early stopping
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Train the optimized model
history_opt = nn_opt.fit(X_train_scaled, y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping], verbose=1)

# Evaluate the optimized model
model_loss_opt, model_accuracy_opt = nn_opt.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Optimized Loss: {model_loss_opt}, Optimized Accuracy: {model_accuracy_opt}")

Epoch 1/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 708us/step - accuracy: 0.6797 - loss: 0.6262 - val_accuracy: 0.7307 - val_loss: 0.5584
Epoch 2/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 590us/step - accuracy: 0.7153 - loss: 0.5794 - val_accuracy: 0.7374 - val_loss: 0.5560
Epoch 3/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 588us/step - accuracy: 0.7278 - loss: 0.5584 - val_accuracy: 0.7369 - val_loss: 0.5529
Epoch 4/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step - accuracy: 0.7285 - loss: 0.5601 - val_accuracy: 0.7387 - val_loss: 0.5501
Epoch 5/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 591us/step - accuracy: 0.7291 - loss: 0.5564 - val_accuracy: 0.7365 - val_loss: 0.5492
Epoch 6/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step - accuracy: 0.7219 - loss: 0.5620 - val_accuracy: 0.7394 - val_loss: 0.5467
Epoch 7/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 580us/step - accuracy: 0.7243 - loss: 0.5600 - val_accuracy: 0.7374 - val_loss: 0.5464
Epoch 8/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 586us/step - accuracy: 0.7301 - loss: 0

In [8]:

# Save the optimized model
nn_opt.save("AlphabetSoupCharity_Optimization.h5")